In [ ]:
pip install comet-ml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import copy
import pdb
import itertools
import random
import string
import math
from pathlib import Path
from collections import OrderedDict

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torchvision.datasets import MNIST
import torch.nn.utils.weight_norm as weightNorm
from torch.nn.functional import relu, avg_pool2d
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data.sampler import Sampler, RandomSampler
import torchvision.transforms.functional as TorchVisionFunc
from torch.utils.data import TensorDataset, DataLoader, Dataset, ConcatDataset, Subset

from comet_ml import Experiment

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
def save_model(model, path):
    torch.save(model.cpu(), path)

def load_model(path):
    model = torch.load(path)
    return model

def setup_experiment(experiment, config):
    Path(f"./checkpoints/{config['dataset']}").mkdir(parents=True, exist_ok=True)
    init_model = ResNet18(config=config) if 'cifar' in config['dataset'] else MLP(config)
    save_model(init_model, f"./checkpoints/{config['dataset']}/init.pth")
    experiment.log_parameters(config)

# Compute grads

In [ ]:
def flatten_grads(m, numpy_output=False, bias=True, only_linear=False):
    total_grads = []
    for name, param in m.named_parameters():
        if only_linear:
            if (bias or not 'bias' in name) and 'linear' in name:
                total_grads.append(param.grad.detach().view(-1))
        else:
            if (bias or not 'bias' in name) and not 'bn' in name and not 'IC' in name:
                try:
                    total_grads.append(param.grad.detach().view(-1))
                except AttributeError:
                    pass

    total_grads = torch.cat(total_grads)
    if numpy_output:
        return total_grads.cpu().detach().numpy()
    return total_grads


def compute_and_flatten_example_grads(m, criterion, data, target, task_id):
    _eg = []
    criterion2 = nn.CrossEntropyLoss(reduction='none').to(DEVICE)
    m.eval()
    m.zero_grad()
    pred = m(data, task_id)
    loss = criterion2(pred, target)
    for idx in range(len(data)):
        loss[idx].backward(retain_graph=True)
        _g = flatten_grads(m, numpy_output=True)
        _eg.append(torch.Tensor(_g))
        m.zero_grad()
    return torch.stack(_eg)

def flatten_example_grads(m, numpy_output=False):
    total_grads = []
    for param in m.parameters():
            total_grads.append(param.grad1.view(param.grad1.size()[0], -1))
    total_grads = torch.cat(total_grads, 1)
    if numpy_output:
        return total_grads.cpu().detach().numpy()
    return total_grads

# Coreset Selection

In [ ]:
def sample_selection(g, eg, config, ref_grads=None, attn=None):
    ng = torch.norm(g)
    neg = torch.norm(eg, dim=1)
    mean_sim = torch.matmul(g,eg.t()) / torch.maximum(ng*neg, torch.ones_like(neg)*1e-6)
    negd = torch.unsqueeze(neg, 1)

    cross_div = torch.matmul(eg,eg.t()) / torch.maximum(torch.matmul(negd, negd.t()), torch.ones_like(negd)*1e-6)
    avg_div = torch.mean(cross_div, 0)

    coreset_aff = 0.
    if ref_grads is not None:
        ref_ng = torch.norm(ref_grads)
        coreset_aff = torch.matmul(ref_grads, eg.t()) / torch.maximum(ref_ng*neg, torch.ones_like(neg)*1e-6)

    measure = mean_sim - avg_div + config['tau'] * coreset_aff
    _, u_idx = torch.sort(measure, descending=True)
    return u_idx.cpu().numpy()

In [ ]:
def classwise_fair_selection(task, cand_target, sorted_index, num_per_label, config, is_shuffle=True):
    num_examples_per_task = config['memory_size'] // task
    num_examples_per_class = num_examples_per_task // config['n_classes']
    num_residuals = num_examples_per_task - num_examples_per_class * config['n_classes']
    residuals =  np.sum([(num_examples_per_class - n_c)*(num_examples_per_class > n_c) for n_c in num_per_label])
    num_residuals += residuals

    # Get the number of coreset instances per class
    while True:
        n_less_sample_class =  np.sum([(num_examples_per_class > n_c) for n_c in num_per_label])
        num_class = (config['n_classes']-n_less_sample_class)
        if (num_residuals // num_class) > 0:
            num_examples_per_class += (num_residuals // num_class)
            num_residuals -= (num_residuals // num_class) * num_class
        else:
            break
    # Get best coresets per class
    selected = []
    target_tid = np.floor(max(cand_target)/config['n_classes'])

    for j in range(config['n_classes']):
        position = np.squeeze((cand_target[sorted_index]==j+(target_tid*config['n_classes'])).nonzero())
        if position.numel() > 1:
            selected.append(position[:num_examples_per_class])
        elif position.numel() == 0:
            continue
        else:
            selected.append([position])
    # Fill rest space as best residuals
    selected = np.concatenate(selected)
    unselected = np.array(list(set(np.arange(num_examples_per_task))^set(selected)))
    final_num_residuals = num_examples_per_task - len(selected)
    best_residuals = unselected[:final_num_residuals]
    selected = np.concatenate([selected, best_residuals])

    if is_shuffle:
        random.shuffle(selected)

    return sorted_index[selected.astype(int)]

In [ ]:
def select_coreset(loader, task, model, candidates, config, candidate_size=250, fair_selection=True):
    criterion = nn.CrossEntropyLoss().to(DEVICE)
    temp_optimizer = torch.optim.SGD(model.parameters(), lr=config['seq_lr'], momentum=config['momentum'])
    temp_optimizer.zero_grad()

    if fair_selection:
        # collect candidates
        cand_data, cand_target = [], []
        cand_size = len(candidates)
        for batch_idx, (data, target, task_id) in enumerate(loader['sequential'][task]['train']):
            if batch_idx == cand_size:
                break
            try:
                cand_data.append(data[candidates[batch_idx]])
                cand_target.append(target[candidates[batch_idx]])
            except IndexError:
                pass
        cand_data = torch.cat(cand_data, 0)
        cand_target = torch.cat(cand_target, 0)

        random_pick_up = torch.randperm(len(cand_target))[:candidate_size]
        cand_data = cand_data[random_pick_up]
        cand_target = cand_target[random_pick_up]

        num_per_label = [len((cand_target==(jj+config['n_classes']*(task-1))).nonzero()) for jj in range(config['n_classes'])]

        num_examples_per_task = config['memory_size'] // task
        pred = model(cand_data.to(DEVICE), task)
        loss = criterion(pred, cand_target.long().to(DEVICE))
        loss.backward()

        # Coreset update
        _eg = compute_and_flatten_example_grads(model, criterion, cand_data.to(DEVICE), cand_target.long().to(DEVICE), task)
        _g = torch.mean(_eg, 0)
        sorted = sample_selection(_g, _eg, config)

        pick = torch.randperm(len(sorted))
        selected = classwise_fair_selection(task, cand_target, pick, num_per_label, config, is_shuffle=True)

        loader['coreset'][task]['train'].data = copy.deepcopy(cand_data[selected])
        loader['coreset'][task]['train'].targets = copy.deepcopy(cand_target[selected])
        num_per_label = [len((cand_target[selected]==(jj+config['n_classes']*(task-1))).nonzero()) for jj in range(config['n_classes'])]
    else:
        pass


In [ ]:
def reconstruct_coreset_loader(config, dataset, task):
    trains = []
    all_coreset = {}
    for tid in range(1, task+1):
        num_examples_per_task = config['memory_size'] // task
        coreset = Coreset(num_examples_per_task, input_shape=[3, 32, 32])

        pick_idx = torch.randperm(num_examples_per_task)
        coreset.data = copy.deepcopy(dataset[tid]['train'].data[pick_idx])
        coreset.targets = copy.deepcopy(dataset[tid]['train'].targets[pick_idx])
        coreset_loader = torch.utils.data.DataLoader(coreset, batch_size=config['batch_size'], shuffle=False, num_workers=0, pin_memory=True)
        train_loader = fast_cifar_loader(coreset_loader, tid, eval=False)
        if ('cifar' in config['dataset']):
            train_loader = fast_cifar_loader(coreset_loader, tid, eval=False)
        else:
            train_loader = fast_mnist_loader(coreset_loader, eval=False)
        trains += train_loader
    all_coreset = random.sample(trains[:], len(trains))
    return all_coreset


def get_coreset_loss(model, iterloader, config):
    criterion = nn.CrossEntropyLoss().to(DEVICE)
    model.train()
    coreset_loss = 0
    count = 0
    data, target, task_id = iterloader
    count += len(target)
    data = data.to(DEVICE)
    target = target.to(DEVICE)
    output = model(data, task_id)
    coreset_loss += criterion(output, target.long())
    coreset_loss /= count
    return coreset_loss


In [ ]:
def update_coreset(loader, task, model, task_id, config):
    # Coreset update
    num_examples_per_task = config['memory_size'] // task
    prv_nept = config['memory_size'] // (task-1)

    for tid in range(1, task):
        criterion = nn.CrossEntropyLoss().to(DEVICE)
        temp_optimizer = torch.optim.SGD(model.parameters(), lr=config['seq_lr'], momentum=config['momentum'])

        tid_coreset = loader['coreset'][tid]['train'].data
        tid_targets = loader['coreset'][tid]['train'].targets

        temp_optimizer.zero_grad()

        pred = model(tid_coreset.to(DEVICE), task_id)
        loss = criterion(pred, tid_targets.long().to(DEVICE))
        loss.backward()
        _tid_eg = compute_and_flatten_example_grads(model, criterion, tid_coreset.to(DEVICE), tid_targets.to(DEVICE), tid)
        _tid_g = torch.mean(_tid_eg, 0)
        pick = sample_selection(_tid_g, _tid_eg, config)

        class_idx = [tid_targets.cpu().numpy() == i for i in range(config['n_classes'])]
        num_per_label = [len((tid_targets.cpu()==(jj+config['n_classes']*(task-1))).nonzero()) for jj in range(config['n_classes'])]

        selected = classwise_fair_selection(task, tid_targets, pick, num_per_label, config)
        _nn = [len((tid_targets[selected]==(jj+config['n_classes']*(tid-1))).nonzero()) for jj in range(config['n_classes'])]

        loader['coreset'][tid]['train'].data = copy.deepcopy(loader['coreset'][tid]['train'].data[selected])
        loader['coreset'][tid]['train'].targets = copy.deepcopy(loader['coreset'][tid]['train'].targets[selected])


# Train

In [ ]:
def train_single_step(model, optimizer, loader, task, step, config):
    criterion = nn.CrossEntropyLoss().to(DEVICE)
    is_last_step = True if step == config['n_substeps'] else False
    candidates_indices=[]
    for batch_idx, (data, target, task_id) in enumerate(loader['sequential'][task]['train']):
        model.train()
        data = data.to(DEVICE)
        target = target.to(DEVICE)
        is_ocspick = True if (config['ocspick'] and len(data) > config['batch_size']) else False
        optimizer.zero_grad()
        if is_ocspick and step != 1:
            _eg = compute_and_flatten_example_grads(model, criterion, data, target, task_id)
            _g = torch.mean(_eg, 0)
            sorted = sample_selection(_g, _eg, config)
            pick = sorted[:config['batch_size']]
            optimizer.zero_grad()
            pred = model(data[pick], task_id)
            loss = criterion(pred, target[pick])
            loss.backward()

            # Select coresets at final step
            if is_last_step:
                candidates_indices.append(pick)
        else:
            size = min(len(data), config['batch_size'])
            pick = torch.randperm(len(data))[:size]
            pred = model(data[pick], task_id)
            loss = criterion(pred, target[pick])
            loss.backward()
        optimizer.step()

    if is_last_step:
        select_coreset(loader, task, model, candidates_indices, config)
    return model


In [ ]:
def train_ocs_single_step(model, optimizer, loader, task, step, config):
    criterion = nn.CrossEntropyLoss().to(DEVICE)
    is_last_step = step == config['n_substeps']

    # Concatenate previous task data and targets
    prev_coreset = [loader['coreset'][tid]['train'].data for tid in range(1, task)]
    prev_targets = [loader['coreset'][tid]['train'].targets for tid in range(1, task)]
    c_x = torch.cat(prev_coreset, 0)
    c_y = torch.cat(prev_targets, 0)

    ref_loader = reconstruct_coreset_loader(config, loader['coreset'], task - 1)
    ref_iterloader = itertools.cycle(ref_loader)  # Create an iterator that cycles through ref_loader

    candidates_indices = []

    for batch_idx, (data, target, task_id) in enumerate(loader['sequential'][task]['train']):
        model.eval()
        optimizer.zero_grad()
        is_rand_start = step == 1

        # Compute reference gradients
        ref_pred = model(c_x.to(DEVICE), task)
        ref_loss = criterion(ref_pred, c_y.long().to(DEVICE))
        ref_loss.backward()
        ref_grads = copy.deepcopy(flatten_grads(model))
        optimizer.zero_grad()

        data = data.to(DEVICE)
        target = target.to(DEVICE)

        if is_rand_start:
            size = min(len(data), config['batch_size'])
            pick = torch.randperm(len(data))[:size]
        else:
            # Compute example gradients and select samples
            _eg = compute_and_flatten_example_grads(model, criterion, data, target, task_id)
            _g = torch.mean(_eg, 0)
            sorted_indices = sample_selection(_g.cuda(), _eg.cuda(), config, ref_grads=ref_grads)
            pick = sorted_indices[:config['batch_size']]

        model.train()
        optimizer.zero_grad()
        pred = model(data[pick], task_id)
        loss = criterion(pred, target[pick])

        ref_data = next(ref_iterloader)
        ref_loss = get_coreset_loss(model, ref_data, config)
        loss += config['ref_hyp'] * ref_loss
        loss.backward()
        optimizer.step()

        if is_last_step:
            candidates_indices.append(pick)

    if is_last_step:
        select_coreset(loader, task, model, candidates_indices, config)
        update_coreset(loader, task, model, task_id, config)

    return model

In [ ]:
def eval_single_epoch(model, loader, config):
    model.to(DEVICE)
    model.eval()

    criterion = nn.CrossEntropyLoss().to(DEVICE)
    total_loss, correct, total_samples = 0.0, 0, 0

    class_correct = np.zeros(config['n_classes'])
    class_total = np.zeros(config['n_classes'])

    with torch.no_grad():
        for data, target, task_id in loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data, task_id)
            loss = criterion(output, target)

            total_loss += loss.item() * len(target)
            total_samples += len(target)

            preds = output.argmax(dim=1)
            correct += (preds == target).sum().item()

            for cid in range(config['n_classes']):
                cid_index = (target == cid) if 'cifar' not in config['dataset'] else (target == cid + (task_id - 1) * config['n_classes'])
                class_correct[cid] += (preds == target)[cid_index].sum().item()
                class_total[cid] += cid_index.sum().item()

    avg_loss = total_loss / total_samples
    avg_acc = 100.0 * correct / total_samples
    per_class_acc = [round(100.0 * a / b, 4) if b > 0 else 0.0 for a, b in zip(class_correct, class_total)]

    return {'accuracy': avg_acc, 'per_class_accuracy': per_class_acc, 'loss': avg_loss}


In [ ]:
def train_task_sequentially(task, train_loader, config, summary=None):
    current_lr = config['seq_lr'] * (config['lr_decay'])**(task-1)
    prev_model_path = f"./checkpoints/{config['dataset']}/init.pth"
    model = load_model(prev_model_path).to(DEVICE)
    optimizer = torch.optim.SGD(model.parameters(), lr=current_lr, momentum=config['momentum'])
    config['n_substeps'] = int((config['stream_size'] / config['batch_size']))
    for _step in range(1, config['n_substeps']+1):
        if task == 1 or (config['ocspick'] == False):
            model = train_single_step(model, optimizer, train_loader, task, _step, config)
        else:
            model = train_ocs_single_step(model, optimizer, train_loader, task, _step, config)
        metrics = eval_single_epoch(model, train_loader['sequential'][task]['val'], config)
        print('Epoch {} >> (per-task accuracy): {}'.format(_step/config['n_substeps'], np.mean(metrics['accuracy'])))
        print('Epoch {} >> (class accuracy): {}'.format(_step/config['n_substeps'], metrics['per_class_accuracy']))
    return model


# Model

In [ ]:

class MLP(nn.Module):
    """
    Two layer MLP for MNIST benchmarks.
    """
    def __init__(self, config):
        super(MLP, self).__init__()
        self.save_acts = False
        self.acts = {}
        self.config = config
        self.W1 = nn.Linear(784, config['mlp_hiddens'])
        self.dropout_1 =  nn.Dropout(p=config['dropout'])
        self.relu = nn.ReLU(inplace=True)
        self.W2 = nn.Linear(config['mlp_hiddens'], config['mlp_hiddens'])
        self.dropout_2 =  nn.Dropout(p=config['dropout'])

        self.W3 = nn.Linear(config['mlp_hiddens'], 10)
        # self.dropout_p = config['dropout']

    def embed(self, x):
        out = self.W1(x)
        out = self.relu(out)

        if self.save_acts:
            self.acts['layer 1'] = out.detach().clone()

        if self.config['dropout'] > 0:
            out = self.dropout_1(out)
        out = self.W2(out)
        self.feature = self.relu(out)
        if self.save_acts:
            self.acts['layer 2'] = self.feature.detach().clone()
        if self.config['dropout'] > 0:
            out = self.dropout_2(self.feature)
        return out


    def forward(self, x, task_id=None):
        # x = x.view(-1, 784 + self.num_condition_neurons)
        out = self.embed(x)
        out = self.W3(self.feature)
        # out = nn.functional.dropout(out, p=self.dropout_p)
        return out

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, config={}):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.conv2 = conv3x3(planes, planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1,
                          stride=stride, bias=False),
            )
        self.IC1 = nn.Sequential(
            nn.BatchNorm2d(planes, track_running_stats=False),
            nn.Dropout(p=config['dropout'])
            )

        self.IC2 = nn.Sequential(
            nn.BatchNorm2d(planes, track_running_stats=False),
            nn.Dropout(p=config['dropout'])
            )

    def forward(self, x):
        out = self.conv1(x)
        out = relu(out)
        out = self.IC1(out)

        out += self.shortcut(x)
        out = relu(out)
        out = self.IC2(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes, nf, config={}):
        super(ResNet, self).__init__()
        self.in_planes = nf
        self.conv1 = conv3x3(3, nf * 1)
        self.bn1 = nn.BatchNorm2d(nf * 1, track_running_stats=False)
        self.layer1 = self._make_layer(block, nf * 1, num_blocks[0], stride=1, config=config)
        self.layer2 = self._make_layer(block, nf * 2, num_blocks[1], stride=2, config=config)
        self.layer3 = self._make_layer(block, nf * 4, num_blocks[2], stride=2, config=config)
        self.layer4 = self._make_layer(block, nf * 8, num_blocks[3], stride=2, config=config)
        self.linear = nn.Linear(nf * 8 * block.expansion, num_classes)
        self.config =config

    def _make_layer(self, block, planes, num_blocks, stride, config):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, config=config))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def embed(self, x):
        bsz = x.size(0)
        out = relu(self.bn1(self.conv1(x.view(bsz, 3, 32, 32))))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        return out

    def forward(self, x, task_id):
        out = self.embed(x)
        out = self.linear(out)
        t = task_id
        if isinstance(self.config['n_classes'], int):
            offset1 = int((t-1) * 5)
            offset2 = int(t * 5)
            if offset1 > 0:
                out[:, :offset1].data.fill_(-10e10)
            if offset2 < 100:
                out[:, offset2:100].data.fill_(-10e10)
            return out
        else:
            offsets = [sum(self.config['n_classes'][:c]) for c in range(1,len(self.config['n_classes'])+1)]
            offset1 = int(offsets[t-1])
            offset2 = int(offsets[t])
            if offset1 > 0:
                out[:, :offset1].data.fill_(-10e10)
            if offset2 < offsets[-1]:
                out[:, offset2:offsets[-1]].data.fill_(-10e10)
            return out

def ResNet18(nclasses=100, nf=20, config={}):
    net = ResNet(BasicBlock, [2, 2, 2, 2], nclasses, nf, config=config)
    return net

# ِDataset


# MNIST

In [ ]:
class RotationTransform:
    def __init__(self, angle):
        self.angle = angle

    def __call__(self, x):
        return TorchVisionFunc.rotate(x, self.angle, fill=(0,))

def fast_mnist_loader(loaders, eval=True, device='cpu'):
    trains, evals = [], []
    if eval:
        train_loader, eval_loader = loaders
        for data, target in train_loader:
            data = data.to(device).view(-1, 784)
            target = target.to(device)
            trains.append([data, target, None])

        for data, target in eval_loader:
            data = data.to(device).view(-1, 784)
            target = target.to(device)
            evals.append([data, target, None])
        return trains, evals
    else:
        train_loader = loaders

        for data, target in train_loader:
            data = data.to(device).view(-1, 784)
            target = target.to(device)
            trains.append([data, target, None])
        return trains

def get_balanced_noisy_rotated_mnist(task_id, batch_size, per_task_rotation):
    print('Noisy Rotated MNIST')
    rotation_degree = (task_id - 1)*per_task_rotation

    train_transforms = torchvision.transforms.Compose([
        RotationTransform(rotation_degree),
        torchvision.transforms.ToTensor(),
        ])

    test_transforms = torchvision.transforms.Compose([
        RotationTransform(rotation_degree),
        torchvision.transforms.ToTensor(),
        ])

    train_dataset = MNIST('./data/', train=True, download=True, transform=train_transforms)
    test_dataset = MNIST('./data/', train=False, download=True, transform=test_transforms)
    end_idx = int(len(train_dataset.data) * 0.6)
    shuffle = torch.randperm(len(train_dataset.data))
    idx = shuffle[:end_idx].long()
    for i in idx:
        train_dataset.data[i] = torch.randn(train_dataset.data[i].size())

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(MNIST('./data/', train=False, download=True, transform=test_transforms),  batch_size=256, shuffle=False, num_workers=0, pin_memory=True)

    return train_loader, test_loader


def get_rotated_mnist(task_id, batch_size, per_task_rotation):
    print('Rotated MNIST')
    rotation_degree = (task_id - 1)*per_task_rotation

    transforms = torchvision.transforms.Compose([
        RotationTransform(rotation_degree),
        torchvision.transforms.ToTensor(),
        ])

    train_loader = torch.utils.data.DataLoader(MNIST('./data/', train=True, download=True, transform=transforms), batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    test_loader = torch.utils.data.DataLoader(MNIST('./data/', train=False, download=True, transform=transforms),  batch_size=256, shuffle=False, num_workers=0, pin_memory=True)

    return train_loader, test_loader



# CIFAR100

In [ ]:
def fast_cifar_loader(loaders, task_id, eval=True, device='cpu'):
    trains, evals = [], []
    if eval:
        train_loader, eval_loader = loaders
        for data, target in train_loader:
            data = data.to(device)
            target = target.to(device)
            trains.append([data, target, task_id])

        for data, target in eval_loader:
            data = data.to(device)
            target = target.to(device)
            evals.append([data, target, task_id])
        return trains, evals
    else:
        for data, target in loaders:
            data = data.to(device)
            target = target.to(device)
            trains.append([data, target, task_id])
        return trains

import torch
import numpy as np
from torch.utils.data import DataLoader, Subset

def get_split_cifar100(task_id, batch_size, cifar_train, cifar_test):
    # Calculate class range for the task
    start_class = (task_id - 1) * 5
    end_class = task_id * 5

    # Get indices of samples belonging to the current task's classes
    train_indices = np.where((np.array(cifar_train.targets) >= start_class) &
                             (np.array(cifar_train.targets) < end_class))[0]

    test_indices = np.where((np.array(cifar_test.targets) >= start_class) &
                            (np.array(cifar_test.targets) < end_class))[0]

    # Create DataLoader for the training and testing subsets
    train_loader = DataLoader(Subset(cifar_train, train_indices),
                              batch_size=batch_size, shuffle=True)

    test_loader = DataLoader(Subset(cifar_test, test_indices),
                             batch_size=batch_size, shuffle=True)

    return train_loader, test_loader


def get_imbalanced_split_cifar100(task_id, batch_size, cifar_train, cifar_test):
    # Predefined shuffle of classes to introduce imbalance
    class_shuffle = [25, 11, 23, 76, 12, 30, 62,  6, 89, 44, 84, 29, 82,  3, 10, 24, 64, 72,
                     21,  8, 63, 71, 68, 74,  5, 86, 22, 58, 95, 19, 47, 54, 56,  2, 20, 96,
                     57, 38, 80, 66,  1, 59, 16, 97, 18, 73, 31, 77, 99, 15, 46, 27, 83, 40,
                     48, 88, 33, 36, 81, 55, 85, 14, 13,  7, 65, 50, 78, 43, 91,  4, 69, 52,
                     41, 94, 34, 51, 37,  9, 90, 35, 92, 26, 42,  0, 17, 87, 53, 93, 32, 28,
                     75, 67, 49, 79, 61, 60, 70, 98, 45, 39]

    n_class = 100
    img_max = len(cifar_train.targets) // n_class
    imb_factor = 1 / 10.

    # Create imbalanced indices for training data
    indices_per_class = []
    for class_number in range(n_class):
        class_indices = np.where(np.array(cifar_train.targets) == class_shuffle[class_number])[0]
        num_samples = int(img_max * (imb_factor ** (class_number / (n_class - 1))))
        indices_per_class.append(class_indices[:num_samples])

    imbalanced_indices = np.concatenate(indices_per_class)
    np.random.shuffle(imbalanced_indices)

    # Create a new imbalanced CIFAR-100 training set
    cifar_train_new = copy.deepcopy(cifar_train)
    cifar_train_new.data = cifar_train.data[imbalanced_indices]
    cifar_train_new.targets = [cifar_train.targets[i] for i in imbalanced_indices]

    # Determine class range for the current task
    start_class = (task_id - 1) * 5
    end_class = task_id * 5

    # Get indices for the current task's classes
    train_indices = [i for i, target in enumerate(cifar_train_new.targets) if start_class <= target < end_class]
    test_indices = [i for i, target in enumerate(cifar_test.targets) if start_class <= target < end_class]

    # Create DataLoaders for the current task
    train_loader = DataLoader(Subset(cifar_train_new, train_indices), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(Subset(cifar_test, test_indices), batch_size=batch_size, shuffle=True)

    print('Imbalanced SplitCifar100')
    if task_id == 1:
        print('Number of instances per class:', [len(indices) for indices in indices_per_class])
    current_task_targets = np.array(cifar_train_new.targets)[train_indices]
    print('Number of instances per class for task {}: {}'.format(
        task_id, [np.sum(current_task_targets == c) for c in range(start_class, end_class)]
    ))

    return train_loader, test_loader


# Load Dataset

In [ ]:
class Coreset(torch.utils.data.Dataset):
    def __init__(self, set_size, input_shape=[784]):
        data_shape = [set_size] + input_shape

        self.data = torch.zeros(data_shape)
        self.targets = torch.ones((set_size)) * -1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.targets[idx]
        return x, y


def get_all_loaders(config, per_task_rotation=9, is_coreset=True):

    dataset = config['dataset'].lower()
    loaders = {'sequential': {}, 'coreset': {}}
    class_arr = range(0, 100)

    cifar_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),])
    cifar_train = torchvision.datasets.CIFAR100('./data/', train=True, download=True, transform=cifar_transforms)
    cifar_test = torchvision.datasets.CIFAR100('./data/', train=False, download=True, transform=cifar_transforms)
    if is_coreset:
        print('loading coreset placeholder {}'.format(dataset))
        for task in range(1, config['num_tasks']+1):
            loaders['sequential'][task], loaders['coreset'][task] = {}, {}
            print("loading {} for task {}".format(dataset, task))
            num_example_per_task = config['memory_size']//task
            classes = np.random.choice(class_arr, 5, replace=False)
            if 'mnist' in dataset:
              if 'noisy' in dataset:
                  seq_loader_train , seq_loader_val = fast_mnist_loader(get_balanced_noisy_rotated_mnist(task, config['stream_size'], per_task_rotation), 'cpu')
              else:
                  seq_loader_train , seq_loader_val = fast_mnist_loader(get_rotated_mnist(task, config['stream_size'], per_task_rotation), 'cpu')
              loaders['coreset'][task]['train'] = Coreset(num_example_per_task)
            if 'cifar' in dataset:
              if 'imb' in dataset:
                  seq_loader_train , seq_loader_val = fast_cifar_loader(get_imbalanced_split_cifar100(task, config['stream_size'], cifar_train, cifar_test), task, 'cpu')
              else:
                  seq_loader_train , seq_loader_val = fast_cifar_loader(get_split_cifar100(task, config['stream_size'], cifar_train, cifar_test), task, 'cpu')
              loaders['coreset'][task]['train'] = Coreset(num_example_per_task, [3, 32, 32])
            loaders['sequential'][task]['train'], loaders['sequential'][task]['val'] = seq_loader_train, seq_loader_val

        return loaders
    else:
        # Load sequential tasks


        for task in range(1, config['num_tasks']+1):
            loaders['sequential'][task] = {}
            print("loading {} for task {}".format(dataset, task))
            if 'noisy' in dataset:
                seq_loader_train , seq_loader_val = fast_mnist_loader(get_balanced_noisy_rotated_mnist(task, config['stream_size'], per_task_rotation), 'cpu')
            elif 'mnist' in dataset:
               seq_loader_train , seq_loader_val = fast_mnist_loader(get_rotated_mnist(task, config['stream_size'], per_task_rotation), 'cpu')
            elif 'imb' in dataset:
                seq_loader_train , seq_loader_val = fast_cifar_loader(get_imbalanced_split_cifar100(task, config['stream_size'], cifar_train, cifar_test), task, 'cpu')
            else:
                seq_loader_train , seq_loader_val = fast_cifar_loader(get_split_cifar100(task, config['stream_size'], cifar_train, cifar_test), task, 'cpu')
            seq_loader_train , seq_loader_val = fast_cifar_loader(get_split_cifar100(task, config['stream_size'], cifar_train, cifar_test), task, 'cpu')
            loaders['sequential'][task]['train'], loaders['sequential'][task]['val'] = seq_loader_train, seq_loader_val
        return loaders

In [ ]:
DATASET = 'rot-mnist'
HIDDENS = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

config = {
    'num_tasks': 20,
    'per_task_rotation': 9,
    'memory_size': 200,
    'dataset': DATASET,
    'device': DEVICE,
    'momentum': 0.8,
    'mlp_hiddens': HIDDENS,
    'dropout': 0.2,
    'lr_decay': 0.75 if 'rot-mnist' in DATASET else 0.8,
    'n_classes': 10,
    'seq_lr': 0.005,
    'stream_size': 100,
    'ocspick': True,
    'batch_size': 20,
     'tau': 1000.0,
    'ref_hyp': 10. if 'rot-mnist' in DATASET else 50
}

log_dir =  f"./summery/{config['dataset']}"
summary = SummaryWriter(log_dir)

experiment = Experiment(api_key="hidden_key", project_name="mnist", disabled=True)

loaders = get_all_loaders(config)


def evaluate_model(model, task, loaders, config):
    accuracies, losses = [], []
    for t in range(1, task + 1):
        metrics = eval_single_epoch(model, loaders['sequential'][t]['val'], config)
        accuracies.append(metrics['accuracy'])
        losses.append(metrics['loss'])
        print(f'OCS >> Task {t}: {metrics}')
    return accuracies, losses

def main():
    setup_experiment(experiment, config)

    max_accuracies = [0.0] * config['num_tasks']
    for task in range(1, config['num_tasks'] + 1):
        print(f'---- Task {task} (OCS) ----')
        model = train_task_sequentially(task, loaders, config, summary)

        accuracies, _ = evaluate_model(model, task, loaders, config)
        max_accuracies = [max(acc, max_acc) for acc, max_acc in zip(accuracies, max_accuracies)]

        avg_accuracy = np.mean(accuracies)
        if task > 1:
            forgetting = np.mean(np.array(max_accuracies[:task - 1]) - np.array(accuracies[:task - 1]))/ 100
        else:
            forgetting = 0.0

        print(f"OCS >> (average accuracy): {avg_accuracy}")
        print(f"OCS >> (Forgetting): {forgetting}")
        summary.add_scalar('cl_average_accuracy', avg_accuracy, task - 1)
        print(f'Maximum per-task accuracies: {max_accuracies}\n')

    print(config)
    experiment.end()

if __name__ == "__main__":
    main()


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Files already downloaded and verified
Files already downloaded and verified
loading coreset placeholder rot-mnist
loading rot-mnist for task 1
Rotated MNIST
loading rot-mnist for task 2
Rotated MNIST
loading rot-mnist for task 3
Rotated MNIST
loading rot-mnist for task 4
Rotated MNIST
loading rot-mnist for task 5
Rotated MNIST
loading rot-mnist for task 6
Rotated MNIST
loading rot-mnist for task 7
Rotated MNIST
loading rot-mnist for task 8
Rotated MNIST
loading rot-mnist for task 9
Rotated MNIST
loading rot-mnist for task 10
Rotated MNIST
loading rot-mnist for task 11
Rotated MNIST
loading rot-mnist for task 12
Rotated MNIST
loading rot-mnist for task 13
Rotated MNIST
loading rot-mnist for task 14
Rotated MNIST
loading rot-mnist for task 15
Rotated MNIST
loading rot-mnist for task 16
Rotated MNIST
loading rot-mnist for task 17
Rotated MNIST
loading rot-mnist for task 18
Rotated MNIST
loading rot-mnist for task 19
Rotated MNIST
loading rot-mnist for task 20
Rotated MNIST
---- Task 1 (OC

/tmp/ipykernel_1750/1459498902.py:12: RuntimeWarning: divide by zero encountered in scalar floor_divide
  if (num_residuals // num_class) > 0:


Epoch 1.0 >> (per-task accuracy): 90.03
Epoch 1.0 >> (class accuracy): [97.7551, 98.4141, 85.7558, 93.6634, 88.9002, 81.6143, 93.0063, 89.2996, 78.6448, 91.2785]
OCS >> Task 1: {'accuracy': 89.16, 'per_class_accuracy': [96.9388, 96.7401, 84.1085, 93.3663, 90.3259, 78.2511, 94.1545, 88.9105, 79.5688, 87.3142], 'loss': 0.3896568247616291}
OCS >> Task 2: {'accuracy': 90.03, 'per_class_accuracy': [97.7551, 98.4141, 85.7558, 93.6634, 88.9002, 81.6143, 93.0063, 89.2996, 78.6448, 91.2785], 'loss': 0.34185475299954415}
OCS >> (average accuracy): 89.595
OCS >> (Forgetting): 0.06439999999999997
Maximum per-task accuracies: [95.6]

---- Task 3 (OCS) ----
Epoch 0.2 >> (per-task accuracy): 83.78
Epoch 0.2 >> (class accuracy): [95.8163, 98.1498, 74.2248, 85.4455, 74.4399, 70.0673, 90.8142, 88.1323, 77.5154, 80.1784]
Epoch 0.4 >> (per-task accuracy): 85.25
Epoch 0.4 >> (class accuracy): [97.0408, 96.3877, 73.8372, 89.1089, 87.9837, 81.0538, 87.5783, 80.642, 75.8727, 81.665]
Epoch 0.6 >> (per-task acc

In [ ]:
# NEW
DATASET = 'rot-mnist'
HIDDENS = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

config = {
    'num_tasks': 20,
    'per_task_rotation': 9,
    'memory_size': 200,
    'dataset': DATASET,
    'device': DEVICE,
    'momentum': 0.7,
    'mlp_hiddens': HIDDENS,
    'dropout': 0.15,
    'lr_decay': 0.7 if 'rot-mnist' in DATASET else 0.76,
    'n_classes': 10,
    'seq_lr': 0.006,
    'stream_size': 100,
    'ocspick': True,
    'batch_size': 5,
     'tau': 950.0,
    'ref_hyp': 10. if 'rot-mnist' in DATASET else 50
}

log_dir =  f"./summery/{config['dataset']}"
summary = SummaryWriter(log_dir)

experiment = Experiment(api_key="hidden_key", project_name="mnist", disabled=True)

loaders = get_all_loaders(config)


def evaluate_model(model, task, loaders, config):
    accuracies, losses = [], []
    for t in range(1, task + 1):
        metrics = eval_single_epoch(model, loaders['sequential'][t]['val'], config)
        accuracies.append(metrics['accuracy'])
        losses.append(metrics['loss'])
        print(f'OCS >> Task {t}: {metrics}')
    return accuracies, losses

def main():
    setup_experiment(experiment, config)

    max_accuracies = [0.0] * config['num_tasks']
    for task in range(1, config['num_tasks'] + 1):
        print(f'---- Task {task} (OCS) ----')
        model = train_task_sequentially(task, loaders, config, summary)

        accuracies, _ = evaluate_model(model, task, loaders, config)
        max_accuracies = [max(acc, max_acc) for acc, max_acc in zip(accuracies, max_accuracies)]

        avg_accuracy = np.mean(accuracies)
        if task > 1:
            forgetting = np.mean(np.array(max_accuracies[:task - 1]) - np.array(accuracies[:task - 1]))/ 100
        else:
            forgetting = 0.0

        print(f"OCS >> (average accuracy): {avg_accuracy}")
        print(f"OCS >> (Forgetting): {forgetting}")
        summary.add_scalar('cl_average_accuracy', avg_accuracy, task - 1)
        print(f'Maximum per-task accuracies: {max_accuracies}\n')

    print(config)
    experiment.end()

if __name__ == "__main__":
    main()


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Files already downloaded and verified
Files already downloaded and verified
loading coreset placeholder rot-mnist
loading rot-mnist for task 1
Rotated MNIST
loading rot-mnist for task 2
Rotated MNIST
loading rot-mnist for task 3
Rotated MNIST
loading rot-mnist for task 4
Rotated MNIST
loading rot-mnist for task 5
Rotated MNIST
loading rot-mnist for task 6
Rotated MNIST
loading rot-mnist for task 7
Rotated MNIST
loading rot-mnist for task 8
Rotated MNIST
loading rot-mnist for task 9
Rotated MNIST
loading rot-mnist for task 10
Rotated MNIST
loading rot-mnist for task 11
Rotated MNIST
loading rot-mnist for task 12
Rotated MNIST
loading rot-mnist for task 13
Rotated MNIST
loading rot-mnist for task 14
Rotated MNIST
loading rot-mnist for task 15
Rotated MNIST
loading rot-mnist for task 16
Rotated MNIST
loading rot-mnist for task 17
Rotated MNIST
loading rot-mnist for task 18
Rotated MNIST
loading rot-mnist for task 19
Rotated MNIST
loading rot-mnist for task 20
Rotated MNIST
---- Task 1 (OC

/tmp/ipykernel_19281/1459498902.py:12: RuntimeWarning: divide by zero encountered in scalar floor_divide
  if (num_residuals // num_class) > 0:


Epoch 1.0 >> (per-task accuracy): 91.98
Epoch 1.0 >> (class accuracy): [94.898, 97.9736, 88.3721, 91.0891, 92.4644, 89.3498, 94.2589, 92.0233, 89.5277, 88.999]
OCS >> Task 1: {'accuracy': 90.78, 'per_class_accuracy': [93.3673, 97.533, 85.1744, 89.703, 91.1405, 88.1166, 93.8413, 92.5097, 88.9117, 86.6204], 'loss': 0.38646575004905465}
OCS >> Task 2: {'accuracy': 91.98, 'per_class_accuracy': [94.898, 97.9736, 88.3721, 91.0891, 92.4644, 89.3498, 94.2589, 92.0233, 89.5277, 88.999], 'loss': 0.32343514476362617}
OCS >> (average accuracy): 91.38
OCS >> (Forgetting): 0.07179999999999992
Maximum per-task accuracies: [97.96]

---- Task 3 (OCS) ----
Epoch 0.05 >> (per-task accuracy): 77.47
Epoch 0.05 >> (class accuracy): [96.1224, 97.2687, 67.9264, 69.2079, 84.8269, 83.296, 81.3152, 64.5914, 52.9774, 75.9167]
Epoch 0.1 >> (per-task accuracy): 81.22
Epoch 0.1 >> (class accuracy): [95.4082, 96.9163, 75.1938, 90.396, 88.5947, 71.5247, 88.7265, 66.3424, 71.4579, 65.6095]
Epoch 0.15 >> (per-task accur

In [ ]:
DATASET = 'noisy-rot-mnist'
HIDDENS = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

config = {
    'num_tasks': 20,
    'per_task_rotation': 9,
    'memory_size': 200,
    'dataset': DATASET,
    'device': DEVICE,
    'momentum': 0.8,
    'mlp_hiddens': HIDDENS,
    'dropout': 0.2,
    'lr_decay': 0.75 if 'rot-mnist' in DATASET else 0.8,
    'n_classes': 10,
    'seq_lr': 0.005,
    'stream_size': 100,
    'ocspick': True,
    'batch_size': 20,
     'tau': 1000.0,
    'ref_hyp': 10. if 'rot-mnist' in DATASET else 50
}

log_dir =  f"./summery/{config['dataset']}"
summary = SummaryWriter(log_dir)

experiment = Experiment(api_key="hidden_key", project_name="mnist", disabled=True)

loaders = get_all_loaders(config)


def evaluate_model(model, task, loaders, config):
    accuracies, losses = [], []
    for t in range(1, task + 1):
        metrics = eval_single_epoch(model, loaders['sequential'][t]['val'], config)
        accuracies.append(metrics['accuracy'])
        losses.append(metrics['loss'])
        print(f'OCS >> Task {t}: {metrics}')
    return accuracies, losses

def main():
    setup_experiment(experiment, config)

    max_accuracies = [0.0] * config['num_tasks']
    for task in range(1, config['num_tasks'] + 1):
        print(f'---- Task {task} (OCS) ----')
        model = train_task_sequentially(task, loaders, config, summary)

        accuracies, _ = evaluate_model(model, task, loaders, config)
        max_accuracies = [max(acc, max_acc) for acc, max_acc in zip(accuracies, max_accuracies)]

        avg_accuracy = np.mean(accuracies)
        if task > 1:
            forgetting = np.mean(np.array(max_accuracies[:task - 1]) - np.array(accuracies[:task - 1]))/ 100
        else:
            forgetting = 0.0

        print(f"OCS >> (average accuracy): {avg_accuracy}")
        print(f"OCS >> (Forgetting): {forgetting}")
        summary.add_scalar('cl_average_accuracy', avg_accuracy, task - 1)
        print(f'Maximum per-task accuracies: {max_accuracies}\n')

    print(config)
    experiment.end()

if __name__ == "__main__":
    main()


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Files already downloaded and verified
Files already downloaded and verified
loading coreset placeholder noisy-rot-mnist
loading noisy-rot-mnist for task 1
Noisy Rotated MNIST
loading noisy-rot-mnist for task 2
Noisy Rotated MNIST
loading noisy-rot-mnist for task 3
Noisy Rotated MNIST
loading noisy-rot-mnist for task 4
Noisy Rotated MNIST
loading noisy-rot-mnist for task 5
Noisy Rotated MNIST
loading noisy-rot-mnist for task 6
Noisy Rotated MNIST
loading noisy-rot-mnist for task 7
Noisy Rotated MNIST
loading noisy-rot-mnist for task 8
Noisy Rotated MNIST
loading noisy-rot-mnist for task 9
Noisy Rotated MNIST
loading noisy-rot-mnist for task 10
Noisy Rotated MNIST
loading noisy-rot-mnist for task 11
Noisy Rotated MNIST
loading noisy-rot-mnist for task 12
Noisy Rotated MNIST
loading noisy-rot-mnist for task 13
Noisy Rotated MNIST
loading noisy-rot-mnist for task 14
Noisy Rotated MNIST
loading noisy-rot-mnist for task 15
Noisy Rotated MNIST
loading noisy-rot-mnist for task 16
Noisy Rotated

/tmp/ipykernel_13774/1459498902.py:12: RuntimeWarning: divide by zero encountered in scalar floor_divide
  if (num_residuals // num_class) > 0:


Epoch 1.0 >> (per-task accuracy): 88.0
Epoch 1.0 >> (class accuracy): [93.2653, 98.5022, 82.8488, 86.3366, 86.9654, 86.2108, 92.6931, 91.3424, 75.154, 85.1338]
OCS >> Task 1: {'accuracy': 86.04, 'per_class_accuracy': [92.551, 96.5639, 80.3295, 86.7327, 83.5031, 85.3139, 87.7871, 92.1206, 70.8419, 82.9534], 'loss': 0.4794124701499939}
OCS >> Task 2: {'accuracy': 88.0, 'per_class_accuracy': [93.2653, 98.5022, 82.8488, 86.3366, 86.9654, 86.2108, 92.6931, 91.3424, 75.154, 85.1338], 'loss': 0.40091448981761935}
OCS >> (average accuracy): 87.02000000000001
OCS >> (Forgetting): 0.06819999999999993
Maximum per-task accuracies: [92.86]

---- Task 3 (OCS) ----
Epoch 0.2 >> (per-task accuracy): 65.52
Epoch 0.2 >> (class accuracy): [90.6122, 98.2379, 30.0388, 82.9703, 80.8554, 0.8969, 94.1545, 84.3385, 57.9055, 26.3627]
Epoch 0.4 >> (per-task accuracy): 79.78
Epoch 0.4 >> (class accuracy): [91.3265, 97.6211, 69.186, 80.6931, 86.3544, 58.4081, 89.7704, 84.0467, 63.655, 72.6462]
Epoch 0.6 >> (per-ta

In [ ]:
# NEW
DATASET = 'noisy-rot-mnist'
HIDDENS = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

config = {
    'num_tasks': 20,
    'per_task_rotation': 9,
    'memory_size': 200,
    'dataset': DATASET,
    'device': DEVICE,
    'momentum': 0.7,
    'mlp_hiddens': HIDDENS,
    'dropout': 0.15,
    'lr_decay': 0.7 if 'rot-mnist' in DATASET else 0.76,
    'n_classes': 10,
    'seq_lr': 0.006,
    'stream_size': 100,
    'ocspick': True,
    'batch_size': 5,
     'tau': 950.0,
    'ref_hyp': 10. if 'rot-mnist' in DATASET else 50
}

log_dir =  f"./summery/{config['dataset']}"
summary = SummaryWriter(log_dir)

experiment = Experiment(api_key="hidden_key", project_name="mnist", disabled=True)

loaders = get_all_loaders(config)


def evaluate_model(model, task, loaders, config):
    accuracies, losses = [], []
    for t in range(1, task + 1):
        metrics = eval_single_epoch(model, loaders['sequential'][t]['val'], config)
        accuracies.append(metrics['accuracy'])
        losses.append(metrics['loss'])
        print(f'OCS >> Task {t}: {metrics}')
    return accuracies, losses

def main():
    setup_experiment(experiment, config)

    max_accuracies = [0.0] * config['num_tasks']
    for task in range(1, config['num_tasks'] + 1):
        print(f'---- Task {task} (OCS) ----')
        model = train_task_sequentially(task, loaders, config, summary)

        accuracies, _ = evaluate_model(model, task, loaders, config)
        max_accuracies = [max(acc, max_acc) for acc, max_acc in zip(accuracies, max_accuracies)]

        avg_accuracy = np.mean(accuracies)
        if task > 1:
            forgetting = np.mean(np.array(max_accuracies[:task - 1]) - np.array(accuracies[:task - 1]))/ 100
        else:
            forgetting = 0.0

        print(f"OCS >> (average accuracy): {avg_accuracy}")
        print(f"OCS >> (Forgetting): {forgetting}")
        summary.add_scalar('cl_average_accuracy', avg_accuracy, task - 1)
        print(f'Maximum per-task accuracies: {max_accuracies}\n')

    print(config)
    experiment.end()

if __name__ == "__main__":
    main()


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


Files already downloaded and verified
Files already downloaded and verified
loading coreset placeholder noisy-rot-mnist
loading noisy-rot-mnist for task 1
Noisy Rotated MNIST
loading noisy-rot-mnist for task 2
Noisy Rotated MNIST
loading noisy-rot-mnist for task 3
Noisy Rotated MNIST
loading noisy-rot-mnist for task 4
Noisy Rotated MNIST
loading noisy-rot-mnist for task 5
Noisy Rotated MNIST
loading noisy-rot-mnist for task 6
Noisy Rotated MNIST
loading noisy-rot-mnist for task 7
Noisy Rotated MNIST
loading noisy-rot-mnist for task 8
Noisy Rotated MNIST
loading noisy-rot-mnist for task 9
Noisy Rotated MNIST
loading noisy-rot-mnist for task 10
Noisy Rotated MNIST
loading noisy-rot-mnist for task 11
Noisy Rotated MNIST
loading noisy-rot-mnist for task 12
Noisy Rotated MNIST
loading noisy-rot-mnist for task 13
Noisy Rotated MNIST
loading noisy-rot-mnist for task 14
Noisy Rotated MNIST
loading noisy-rot-mnist for task 15
Noisy Rotated MNIST
loading noisy-rot-mnist for task 16
Noisy Rotated

/tmp/ipykernel_19281/1459498902.py:12: RuntimeWarning: divide by zero encountered in scalar floor_divide
  if (num_residuals // num_class) > 0:


Epoch 1.0 >> (per-task accuracy): 90.35
Epoch 1.0 >> (class accuracy): [95.7143, 96.4758, 94.5736, 95.1485, 77.0876, 78.8117, 93.3194, 86.5759, 90.7598, 92.8642]
OCS >> Task 1: {'accuracy': 86.83, 'per_class_accuracy': [94.0816, 90.7489, 93.5078, 92.3762, 69.5519, 70.1794, 90.501, 86.8677, 87.6797, 90.1883], 'loss': 0.5348783303260803}
OCS >> Task 2: {'accuracy': 90.35, 'per_class_accuracy': [95.7143, 96.4758, 94.5736, 95.1485, 77.0876, 78.8117, 93.3194, 86.5759, 90.7598, 92.8642], 'loss': 0.39027177742123603}
OCS >> (average accuracy): 88.59
OCS >> (Forgetting): 0.10079999999999999
Maximum per-task accuracies: [96.91]

---- Task 3 (OCS) ----
Epoch 0.05 >> (per-task accuracy): 74.82
Epoch 0.05 >> (class accuracy): [96.0204, 92.8634, 76.938, 78.7129, 52.0367, 56.3901, 85.3862, 69.9416, 46.8172, 88.3053]
Epoch 0.1 >> (per-task accuracy): 79.72
Epoch 0.1 >> (class accuracy): [96.4286, 89.4273, 78.3915, 80.495, 55.499, 77.5785, 82.881, 60.8949, 79.9795, 94.5491]
Epoch 0.15 >> (per-task acc